<a href="https://colab.research.google.com/github/ReginaldAboagye/365datascience/blob/master/PROGRESS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
import yfinance as yf
from cvxopt import matrix, solvers
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Step 1: Data Preparation
ticker = "AAPL"
data = yf.download(ticker, start="2008-05-01", end="2009-05-20")
data['Returns'] = data['Adj Close']
data.dropna(inplace=True)
data['Volatility'] = data['Returns'].rolling(window=21).std()
data.dropna(inplace=True)
X = data[['Returns']].values
y = data['Volatility'].values

# Debug: Print initial data stats
print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"Initial X: {X[:5]}")
print(f"Initial y: {y[:5]}")

# Define a function to calculate the Gaussian kernel between two points x and y with a given sigma
def gaussian_kernel(x, y, sigma):
    distance_squared = np.linalg.norm(x - y) ** 2
    return np.exp(-distance_squared / (2 * sigma ** 2))

"""
# Define a Support Vector Regressor (SVR)
class SVM:
    def __init__(self, C=1.0):
        self.C = C  # Regularization parameter
        self.alpha = None  # Lagrange multipliers
        self.b = 0  # Bias term
        self.support_vectors = None  # Support vectors
        self.y_sv = None  # Labels of support vectors

    def fit(self, X, y, sigma_func):
        n_samples, n_features = X.shape
        K = np.zeros((n_samples, n_samples))

        for i in range(n_samples):
            for j in range(n_samples):
                K[i, j] = gaussian_kernel(X[i], X[j], sigma_func(i, j))

        P = matrix(np.outer(y, y) * K)
        q = matrix(-np.ones(n_samples))
        G = matrix(np.vstack((-np.eye(n_samples), np.eye(n_samples))))
        h = matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * self.C)))
        A = matrix(y.reshape(1, -1).astype(float))
        b = matrix(np.zeros(1))

        sol = solvers.qp(P, q, G, h, A, b)
        alpha = np.ravel(sol['x'])

        sv = alpha > 1e-5
        self.alpha = alpha[sv]
        self.support_vectors = X[sv]
        self.y_sv = y[sv]
        self.b = np.mean(self.y_sv - np.sum(self.alpha * self.y_sv * K[sv][:, sv], axis=1))

    def predict(self, X, sigma_func):
        y_pred = []
        for x in X:
            prediction = sum(self.alpha[i] * self.y_sv[i] * gaussian_kernel(x, self.support_vectors[i], sigma_func(i, i))
                             for i in range(len(self.alpha))) + self.b
            y_pred.append(np.sign(prediction))
        return np.array(y_pred)
"""


# Define an Extended Kalman Filter (EKF) class
class EKF:
    def __init__(self, F, H, Q, R):
        self.F = F
        self.H = H
        self.Q = Q
        self.R = R

    def predict(self, x, P):
        x_pred = self.F @ x
        P_pred = self.F @ P @ self.F.T + self.Q
        return x_pred, P_pred

    def update(self, x_pred, P_pred, y):
        y_pred = self.H @ x_pred
        S = self.H @ P_pred @ self.H.T + self.R
        K = P_pred @ self.H.T @ np.linalg.inv(S)
        x_upd = x_pred + K @ (y - y_pred)
        P_upd = P_pred - K @ self.H @ P_pred
        return x_upd, P_upd

    def run(self, y, x0, P0):
        n = len(y)
        x_est = np.zeros((n, len(x0)))
        x_est[0] = x0
        P = P0
        for t in range(1, n):
            x_pred, P_pred = self.predict(x_est[t-1], P)
            x_est[t], P = self.update(x_pred, P_pred, y[t])
        return x_est

# Define a Bayesian Stochastic Volatility EKF-HMC class
class BayesianStochasticVolatilityEKF_HMC:
    def __init__(self, F, H, Q, R, mu_prior, phi_prior, sigma_eta_prior, n_iter=1000, epsilon=0.01, L=10):
        self.ekf = EKF(F, H, Q, R)
        self.mu_prior = mu_prior
        self.phi_prior = phi_prior
        self.sigma_eta_prior = sigma_eta_prior
        self.n_iter = n_iter
        self.epsilon = epsilon
        self.L = L

    def potential_energy(self, params, y):
        mu, phi, sigma_eta = params
        sigma_eta = sigma_eta if np.isscalar(sigma_eta) else sigma_eta.item()
        if sigma_eta <= 0:
            return np.inf
        F = np.array([[phi]])
        H = np.array([[1]])
        Q = np.array([[sigma_eta ** 2]])
        R = np.array([[1]])

        x0 = np.array([mu])
        P0 = np.array([[sigma_eta ** 2]])

        x_est = self.ekf.run(y, x0, P0)

        likelihood = -0.5 * np.sum((y - x_est.flatten()) ** 2)
        prior = -0.5 * (mu - self.mu_prior[0]) ** 2 / self.mu_prior[1] ** 2 \
                -0.5 * (phi - self.phi_prior[0]) ** 2 / self.phi_prior[1] ** 2 \
                -0.5 * (sigma_eta - self.sigma_eta_prior[0]) ** 2 / self.sigma_eta_prior[1] ** 2
        return -(likelihood + prior)

    def gradient_potential_energy(self, params, y):
        mu, phi, sigma_eta = params
        sigma_eta = np.atleast_1d(sigma_eta)[0]

        F = np.array([[phi]])
        H = np.array([[1]])
        Q = np.array([[sigma_eta ** 2]])
        R = np.array([[1]])

        x0 = np.array([mu])
        P0 = np.array([[sigma_eta ** 2]])

        x_est = self.ekf.run(y, x0, P0)

        grad_mu = 0
        grad_phi = 0
        grad_sigma_eta = 0
        for t in range(1, len(y)):
            grad_mu += (y[t] - x_est[t]) / np.exp(x_est[t])
            grad_phi += (y[t] - x_est[t]) * (x_est[t-1] - mu) / np.exp(x_est[t])
            grad_sigma_eta += (y[t] - mu - phi * y[t-1]) ** 2 / (2 * sigma_eta ** 3)

        grad_mu -= (mu - self.mu_prior[0]) / self.mu_prior[1] ** 2
        grad_phi -= (phi - self.phi_prior[0]) / self.phi_prior[1] ** 2
        grad_sigma_eta -= (sigma_eta - self.sigma_eta_prior[0]) / self.sigma_eta_prior[1] ** 2

        return np.array([grad_mu.item(), grad_phi.item(), grad_sigma_eta.item()])

    def leapfrog(self, params, momentum, grad, epsilon):
        momentum_half_step = momentum - 0.5 * epsilon * grad
        params_full_step = params + epsilon * momentum_half_step
        grad_new = self.gradient_potential_energy(params_full_step, log_returns)
        momentum_full_step = momentum_half_step - 0.5 * epsilon * grad_new
        return params_full_step, momentum_full_step, grad_new

    def hmc_step(self, params, y):
        momentum = np.random.normal(size=params.shape)
        current_params = params.copy()
        current_momentum = momentum.copy()
        current_grad = self.gradient_potential_energy(params, y)
        proposed_params = params.copy()
        proposed_momentum = momentum.copy()

        for _ in range(self.L):
            proposed_params, proposed_momentum, current_grad = self.leapfrog(proposed_params, proposed_momentum, current_grad, self.epsilon)

        current_potential = self.potential_energy(current_params, y)
        current_kinetic = 0.5 * np.sum(current_momentum ** 2)
        proposed_potential = self.potential_energy(proposed_params, y)
        proposed_kinetic = 0.5 * np.sum(proposed_momentum ** 2)

        if np.random.uniform() < np.exp(current_potential - proposed_potential + current_kinetic - proposed_kinetic):
            return proposed_params
        else:
            return current_params

    def fit(self, y):
        params = np.array([self.mu_prior[0], self.phi_prior[0], self.sigma_eta_prior[0]])
        samples = np.zeros((self.n_iter, 3))
        for i in range(self.n_iter):
            params = self.hmc_step(params, y)
            samples[i] = params
        return samples

log_returns = np.diff(np.log(data['Returns']))

F = np.array([[0.9]])
H = np.array([[1]])
Q = np.array([[0.01]])
R = np.array([[1]])

hmc_ekf_model = BayesianStochasticVolatilityEKF_HMC(F, H, Q, R, mu_prior=(0, 1), phi_prior=(0.9, 0.1), sigma_eta_prior=(0.1, 0.01), n_iter=1000, epsilon=0.01, L=10)
samples = hmc_ekf_model.fit(log_returns)

# Debug: Print samples summary
print("Posterior mean estimates")
print("mu:", np.mean(samples[:, 0]))
print("phi:", np.mean(samples[:, 1]))
print("sigma_eta:", np.mean(samples[:, 2]))

class HybridModel:
    def __init__(self, C=1.0, mu_prior=(0.0, 1.0), phi_prior=(0.9, 0.1), sigma_eta_prior=(0.1, 0.01), n_iter=1000, epsilon=0.01, L=10):
        self.svm = SVR(kernel="rbf")
        F = np.array([[phi_prior[0]]])
        H = np.array([[1]])
        Q = np.array([[sigma_eta_prior[0] ** 2]])
        R = np.array([[1]])
        self.bsv = BayesianStochasticVolatilityEKF_HMC(F, H, Q, R, mu_prior, phi_prior, sigma_eta_prior, n_iter, epsilon, L)

    def fit(self, X, y, log_returns):
        samples = self.bsv.fit(log_returns)
        h = np.mean(samples, axis=0)
        sigma_func = lambda i, j: np.sqrt(np.exp(h[0] + h[1]))
        self.svm.fit(X, y, sigma_func)

    def predict(self, X):
        samples = self.bsv.fit(np.zeros(len(X)))
        h = np.mean(samples, axis=0)
        sigma_func = lambda i, j: np.sqrt(np.exp(h[0] + h[1]))
        return self.svm.predict(X, sigma_func)

# Instantiate and fit the hybrid model
hybrid_model = HybridModel()
hybrid_model.fit(X, y, log_returns)

# Predictions
data1 = yf.download(ticker, start="2024-05-01", end="2024-06-30")
data1['Returns'] = data1['Adj Close']
data1.dropna(inplace=True)
data1['Volatility'] = data1['Returns'].rolling(window=21).std()
data1.dropna(inplace=True)
X_test = data1[['Returns']].values
y_test = data1['Volatility'].values

hybrid_predictions = hybrid_model.predict(X_test)

# Debug: Check for NaN in predictions
print("Hybrid Predictions: ", hybrid_predictions)
print("NaN in predictions: ", np.isnan(hybrid_predictions).any())



try:
    accuracy = accuracy_score(y_test, hybrid_predictions)
    conf_matrix = confusion_matrix(y_test, hybrid_predictions)
    class_report = classification_report(y_test, hybrid_predictions)

    print("Accuracy:", accuracy)
    print("Confusion Matrix:\n", conf_matrix)
    print("Classification Report:\n", class_report)
-except ValueError as e:
    print("Error in metrics calculation:", e)

[*********************100%%**********************]  1 of 1 completed


X shape: (245, 1), y shape: (245,)
Initial X: [[5.69925642]
 [5.61924076]
 [5.59720039]
 [5.5917654 ]
 [5.71978855]]
Initial y: [0.11027197 0.10549117 0.10158944 0.10152503 0.10482477]
Posterior mean estimates
mu: 0.0
phi: 0.9000000000000002
sigma_eta: 0.10000000000000002


TypeError: float() argument must be a string or a real number, not 'function'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
import yfinance as yf
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Step 1: Data Preparation
ticker = "AAPL"
data = yf.download(ticker, start="2000-01-01", end="2023-12-31")
data['Returns'] = data['Adj Close']
data.dropna(inplace=True)
data['Volatility'] = data['Returns'].rolling(window=21).std()
data.dropna(inplace=True)
X = data[['Returns']].values
y = data['Volatility'].values

# Debug: Print initial data stats
print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"Initial X: {X[:5]}")
print(f"Initial y: {y[:5]}")

# Define an Extended Kalman Filter (EKF) class
class EKF:
    def __init__(self, F, H, Q, R):
        self.F = F
        self.H = H
        self.Q = Q
        self.R = R

    def predict(self, x, P):
        x_pred = self.F @ x
        P_pred = self.F @ P @ self.F.T + self.Q
        return x_pred, P_pred

    def update(self, x_pred, P_pred, y):
        y_pred = self.H @ x_pred
        S = self.H @ P_pred @ self.H.T + self.R
        K = P_pred @ self.H.T @ np.linalg.inv(S)
        x_upd = x_pred + K @ (y - y_pred)
        P_upd = P_pred - K @ self.H @ P_pred
        return x_upd, P_upd

    def run(self, y, x0, P0):
        n = len(y)
        x_est = np.zeros((n, len(x0)))
        x_est[0] = x0
        P = P0
        for t in range(1, n):
            x_pred, P_pred = self.predict(x_est[t-1], P)
            x_est[t], P = self.update(x_pred, P_pred, y[t])
        return x_est

# Define a Bayesian Stochastic Volatility EKF-HMC class
class BayesianStochasticVolatilityEKF_HMC:
    def __init__(self, F, H, Q, R, mu_prior, phi_prior, sigma_eta_prior, n_iter=1000, epsilon=0.01, L=10):
        self.ekf = EKF(F, H, Q, R)
        self.mu_prior = mu_prior
        self.phi_prior = phi_prior
        self.sigma_eta_prior = sigma_eta_prior
        self.n_iter = n_iter
        self.epsilon = epsilon
        self.L = L

    def potential_energy(self, params, y):
        mu, phi, sigma_eta = params
        sigma_eta = sigma_eta if np.isscalar(sigma_eta) else sigma_eta.item()
        if sigma_eta <= 0:
            return np.inf
        F = np.array([[phi]])
        H = np.array([[1]])
        Q = np.array([[sigma_eta ** 2]])
        R = np.array([[1]])

        x0 = np.array([mu])
        P0 = np.array([[sigma_eta ** 2]])

        x_est = self.ekf.run(y, x0, P0)

        likelihood = -0.5 * np.sum((y - x_est.flatten()) ** 2)
        prior = -0.5 * (mu - self.mu_prior[0]) ** 2 / self.mu_prior[1] ** 2 \
                -0.5 * (phi - self.phi_prior[0]) ** 2 / self.phi_prior[1] ** 2 \
                -0.5 * (sigma_eta - self.sigma_eta_prior[0]) ** 2 / self.sigma_eta_prior[1] ** 2
        return -(likelihood + prior)

    def gradient_potential_energy(self, params, y):
        mu, phi, sigma_eta = params
        sigma_eta = np.atleast_1d(sigma_eta)[0]

        F = np.array([[phi]])
        H = np.array([[1]])
        Q = np.array([[sigma_eta ** 2]])
        R = np.array([[1]])

        x0 = np.array([mu])
        P0 = np.array([[sigma_eta ** 2]])

        x_est = self.ekf.run(y, x0, P0)

        grad_mu = 0
        grad_phi = 0
        grad_sigma_eta = 0
        for t in range(1, len(y)):
            grad_mu += (y[t] - x_est[t]) / np.exp(x_est[t])
            grad_phi += (y[t] - x_est[t]) * (x_est[t-1] - mu) / np.exp(x_est[t])
            grad_sigma_eta += (y[t] - mu - phi * y[t-1]) ** 2 / (2 * sigma_eta ** 3)

        grad_mu -= (mu - self.mu_prior[0]) / self.mu_prior[1] ** 2
        grad_phi -= (phi - self.phi_prior[0]) / self.phi_prior[1] ** 2
        grad_sigma_eta -= (sigma_eta - self.sigma_eta_prior[0]) / self.sigma_eta_prior[1] ** 2

        return np.array([grad_mu.item(), grad_phi.item(), grad_sigma_eta.item()])

    def leapfrog(self, params, momentum, grad, epsilon):
        momentum_half_step = momentum - 0.5 * epsilon * grad
        params_full_step = params + epsilon * momentum_half_step
        grad_new = self.gradient_potential_energy(params_full_step, log_returns)
        momentum_full_step = momentum_half_step - 0.5 * epsilon * grad_new
        return params_full_step, momentum_full_step, grad_new

    def hmc_step(self, params, y):
        momentum = np.random.normal(size=params.shape)
        current_params = params.copy()
        current_momentum = momentum.copy()
        current_grad = self.gradient_potential_energy(params, y)
        proposed_params = params.copy()
        proposed_momentum = momentum.copy()

        for _ in range(self.L):
            proposed_params, proposed_momentum, current_grad = self.leapfrog(proposed_params, proposed_momentum, current_grad, self.epsilon)

        current_potential = self.potential_energy(current_params, y)
        current_kinetic = 0.5 * np.sum(current_momentum ** 2)
        proposed_potential = self.potential_energy(proposed_params, y)
        proposed_kinetic = 0.5 * np.sum(proposed_momentum ** 2)

        if np.random.uniform() < np.exp(current_potential - proposed_potential + current_kinetic - proposed_kinetic):
            return proposed_params
        else:
            return current_params

    def fit(self, y):
        params = np.array([self.mu_prior[0], self.phi_prior[0], self.sigma_eta_prior[0]])
        samples = np.zeros((self.n_iter, 3))
        for i in range(self.n_iter):
            params = self.hmc_step(params, y)
            samples[i] = params
        return samples

log_returns = np.diff(np.log(data['Returns']))

F = np.array([[0.9]])
H = np.array([[1]])
Q = np.array([[0.01]])
R = np.array([[1]])

hmc_ekf_model = BayesianStochasticVolatilityEKF_HMC(F, H, Q, R, mu_prior=(0, 1), phi_prior=(0.9, 0.1), sigma_eta_prior=(0.1, 0.01), n_iter=1000, epsilon=0.01, L=10)
samples = hmc_ekf_model.fit(log_returns)

# Debug: Print samples summary
print("Posterior mean estimates")
print("mu:", np.mean(samples[:, 0]))
print("phi:", np.mean(samples[:, 1]))
print("sigma_eta:", np.mean(samples[:, 2]))

class HybridModel:
    def __init__(self, C=1.0, mu_prior=(0.0, 1.0), phi_prior=(0.9, 0.1), sigma_eta_prior=(0.1, 0.01), n_iter=1000, epsilon=0.01, L=10):
        self.svm = SVR(kernel="rbf")
        F = np.array([[phi_prior[0]]])
        H = np.array([[1]])
        Q = np.array([[sigma_eta_prior[0] ** 2]])
        R = np.array([[1]])
        self.bsv = BayesianStochasticVolatilityEKF_HMC(F, H, Q, R, mu_prior, phi_prior, sigma_eta_prior, n_iter, epsilon, L)

    def fit(self, X, y, log_returns):
        samples = self.bsv.fit(log_returns)
        h = np.mean(samples, axis=0)
        sigma_func = lambda i, j: np.sqrt(np.exp(h[0] + h[1]))
        self.svm.fit(X, y)

    def predict(self, X):
        samples = self.bsv.fit(np.zeros(len(X)))
        h = np.mean(samples, axis=0)
        sigma_func = lambda i, j: np.sqrt(np.exp(h[0] + h[1]))
        return self.svm.predict(X)

# Instantiate and fit the hybrid model
hybrid_model = HybridModel()
hybrid_model.fit(X, y, log_returns)

# Predictions
data1 = yf.download(ticker, start="2024-01-01", end="2024-06-30")
data1['Returns'] = data1['Adj Close']
data1.dropna(inplace=True)
data1['Volatility'] = data1['Returns'].rolling(window=21).std()
data1.dropna(inplace=True)
X_test = data1[['Returns']].values
y_test = data1['Volatility'].values

hybrid_predictions = hybrid_model.predict(X_test)

# Debug: Check for NaN in predictions
print("Hybrid Predictions: ", hybrid_predictions)
print("NaN in predictions: ", np.isnan(hybrid_predictions).any())

# Calculate regression metrics
mse = mean_squared_error(y_test, hybrid_predictions)
mae = mean_absolute_error(y_test, hybrid_predictions)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)


[*********************100%%**********************]  1 of 1 completed


X shape: (6017, 1), y shape: (6017,)
Initial X: [[0.75675553]
 [0.74590403]
 [0.77987349]
 [0.81525826]
 [0.8610217 ]]
Initial y: [0.05296062 0.05118237 0.05119596 0.05192094 0.05322455]


In [ ]:
try:
    accuracy = accuracy_score(y_test, hybrid_predictions)
    conf_matrix = confusion_matrix(y_test, hybrid_predictions)
    class_report = classification_report(y_test, hybrid_predictions)

    print("Accuracy:", accuracy)
    print("Confusion Matrix:\n", conf_matrix)
    print("Classification Report:\n", class_report)
except ValueError as e:
    print("Error in metrics calculation:", e)

Error in metrics calculation: continuous is not supported


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
import yfinance as yf
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Step 1: Data Preparation
ticker = "AAPL"  # Ticker symbol for Apple Inc.
data = yf.download(ticker, start="2008-05-01", end="2009-05-20")  # Download historical stock data

if data.empty:  # Check if data is empty
    raise ValueError("No data was downloaded. Please check the ticker symbol or the date range.")  # Raise error if no data

data['Returns'] = data['Adj Close'].pct_change()  # Calculate daily returns
data.dropna(inplace=True)  # Remove missing values
data['Volatility'] = data['Returns'].rolling(window=21).std()  # Calculate rolling volatility
data.dropna(inplace=True)  # Remove missing values

X = data[['Returns']].values  # Features (returns)
y = data['Volatility'].values  # Target (volatility)

# Debug: Print initial data stats
print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"Initial X: {X[:5]}")
print(f"Initial y: {y[:5]}")

# Define a function to calculate the Gaussian kernel between two points x and y with a given sigma
def gaussian_kernel(x, y, sigma):
    distance_squared = np.linalg.norm(x - y) ** 2  # Calculate squared Euclidean distance
    return np.exp(-distance_squared / (2 * sigma ** 2))  # Apply Gaussian function

# Define an Extended Kalman Filter (EKF) class
class EKF:
    def __init__(self, F, H, Q, R):
        self.F = F  # State transition matrix
        self.H = H  # Observation matrix
        self.Q = Q  # Process noise covariance
        self.R = R  # Observation noise covariance

    def predict(self, x, P):
        x_pred = self.F @ x  # Predict the state
        P_pred = self.F @ P @ self.F.T + self.Q  # Predict the error covariance
        return x_pred, P_pred

    def update(self, x_pred, P_pred, y):
        y_pred = self.H @ x_pred  # Predict the observation
        S = self.H @ P_pred @ self.H.T + self.R  # Innovation (residual) covariance
        K = P_pred @ self.H.T @ np.linalg.inv(S)  # Kalman gain
        x_upd = x_pred + K @ (y - y_pred)  # Update the state estimate
        P_upd = P_pred - K @ self.H @ P_pred  # Update the error covariance
        return x_upd, P_upd

    def run(self, y, x0, P0):
        n = len(y)  # Number of observations
        x_est = np.zeros((n, len(x0)))  # Initialize state estimates
        x_est[0] = x0  # Set initial state
        P = P0  # Set initial error covariance
        for t in range(1, n):
            x_pred, P_pred = self.predict(x_est[t-1], P)  # Prediction step
            x_est[t], P = self.update(x_pred, P_pred, y[t])  # Update step
        return x_est

# Define a Bayesian Stochastic Volatility EKF-HMC class
class BayesianStochasticVolatilityEKF_HMC:
    def __init__(self, F, H, Q, R, mu_prior, phi_prior, sigma_eta_prior, n_iter=1000, epsilon=0.01, L=10):
        self.ekf = EKF(F, H, Q, R)  # Initialize EKF
        self.mu_prior = mu_prior  # Prior for mu
        self.phi_prior = phi_prior  # Prior for phi
        self.sigma_eta_prior = sigma_eta_prior  # Prior for sigma_eta
        self.n_iter = n_iter  # Number of HMC iterations
        self.epsilon = epsilon  # Step size for HMC
        self.L = L  # Number of leapfrog steps

    def potential_energy(self, params, y):
        mu, phi, sigma_eta = params
        sigma_eta = sigma_eta if np.isscalar(sigma_eta) else sigma_eta.item()
        if sigma_eta <= 0:
            return np.inf
        F = np.array([[phi]])
        H = np.array([[1]])
        Q = np.array([[sigma_eta ** 2]])
        R = np.array([[1]])

        x0 = np.array([mu])
        P0 = np.array([[sigma_eta ** 2]])

        x_est = self.ekf.run(y, x0, P0)

        likelihood = -0.5 * np.sum((y - x_est.flatten()) ** 2)
        prior = -0.5 * (mu - self.mu_prior[0]) ** 2 / self.mu_prior[1] ** 2 \
                -0.5 * (phi - self.phi_prior[0]) ** 2 / self.phi_prior[1] ** 2 \
                -0.5 * (sigma_eta - self.sigma_eta_prior[0]) ** 2 / self.sigma_eta_prior[1] ** 2
        return -(likelihood + prior)

    def gradient_potential_energy(self, params, y):
        mu, phi, sigma_eta = params
        sigma_eta = np.atleast_1d(sigma_eta)[0]

        F = np.array([[phi]])
        H = np.array([[1]])
        Q = np.array([[sigma_eta ** 2]])
        R = np.array([[1]])

        x0 = np.array([mu])
        P0 = np.array([[sigma_eta ** 2]])

        x_est = self.ekf.run(y, x0, P0)

        grad_mu = 0
        grad_phi = 0
        grad_sigma_eta = 0
        for t in range(1, len(y)):
            grad_mu += (y[t] - x_est[t]) / np.exp(x_est[t])
            grad_phi += (y[t] - x_est[t]) * (x_est[t-1] - mu) / np.exp(x_est[t])
            grad_sigma_eta += (y[t] - mu - phi * y[t-1]) ** 2 / (2 * sigma_eta ** 3)

        grad_mu -= (mu - self.mu_prior[0]) / self.mu_prior[1] ** 2
        grad_phi -= (phi - self.phi_prior[0]) / self.phi_prior[1] ** 2
        grad_sigma_eta -= (sigma_eta - self.sigma_eta_prior[0]) / self.sigma_eta_prior[1] ** 2

        return np.array([grad_mu.item(), grad_phi.item(), grad_sigma_eta.item()])

    def leapfrog(self, params, momentum, grad, epsilon):
        momentum_half_step = momentum - 0.5 * epsilon * grad
        params_full_step = params + epsilon * momentum_half_step
        grad_new = self.gradient_potential_energy(params_full_step, log_returns)
        momentum_full_step = momentum_half_step - 0.5 * epsilon * grad_new
        return params_full_step, momentum_full_step, grad_new

    def hmc_step(self, params, y):
        momentum = np.random.normal(size=params.shape)
        current_params = params.copy()
        current_momentum = momentum.copy()
        current_grad = self.gradient_potential_energy(params, y)
        proposed_params = params.copy()
        proposed_momentum = momentum.copy()

        for _ in range(self.L):
            proposed_params, proposed_momentum, current_grad = self.leapfrog(proposed_params, proposed_momentum, current_grad, self.epsilon)

        current_potential = self.potential_energy(current_params, y)
        current_kinetic = 0.5 * np.sum(current_momentum ** 2)
        proposed_potential = self.potential_energy(proposed_params, y)
        proposed_kinetic = 0.5 * np.sum(proposed_momentum ** 2)

        if np.random.uniform() < np.exp(current_potential - proposed_potential + current_kinetic - proposed_kinetic):
            return proposed_params
        else:
            return current_params

    def fit(self, y):
        if len(y) == 0:
            raise ValueError("Input data is empty. Cannot fit the model.")

        params = np.array([self.mu_prior[0], self.phi_prior[0], self.sigma_eta_prior[0]])
        samples = np.zeros((self.n_iter, 3))
        for i in range(self.n_iter):
            params = self.hmc_step(params, y)
            samples[i] = params
        return samples

log_returns = np.diff(np.log(data['Returns']))  # Calculate log returns

F = np.array([[0.9]])  # State transition matrix
H = np.array([[1]])  # Observation matrix
Q = np.array([[0.01]])  # Process noise covariance
R = np.array([[1]])  # Observation noise covariance

# Fit Bayesian Stochastic Volatility EKF-HMC model
hmc_ekf_model = BayesianStochasticVolatilityEKF_HMC(F, H, Q, R, mu_prior=(0, 1), phi_prior=(0.9, 0.1), sigma_eta_prior=(0.1, 0.01), n_iter=1000, epsilon=0.01, L=10)
samples = hmc_ekf_model.fit(log_returns)

# Debug: Print samples summary
print("Samples summary:")
print(samples)

# Define a hybrid model that combines SVM with Bayesian Stochastic Volatility EKF-HMC
class HybridModel:
    def __init__(self):
        self.svm = SVR(kernel='rbf')  # Initialize SVR with RBF kernel
        F = np.array([[0.9]])
        H = np.array([[1]])
        Q = np.array([[0.01]])
        R = np.array([[1]])
        self.bsv = BayesianStochasticVolatilityEKF_HMC(F, H, Q, R, mu_prior=(0, 1), phi_prior=(0.9, 0.1), sigma_eta_prior=(0.1, 0.01), n_iter=1000, epsilon=0.01, L=10)

    def fit(self, X, y, log_returns):
        samples = self.bsv.fit(log_returns)  # Fit the Bayesian Stochastic Volatility EKF-HMC model
        h = np.mean(samples, axis=0)  # Calculate mean of samples
        sigma_weights = np.sqrt(np.exp(h[0] + h[1]))  # Define sigma weights
        self.svm.fit(X, y, sample_weight=sigma_weights)  # Fit the SVR model with sigma weights

    def predict(self, X):
        samples = self.bsv.fit(np.zeros(len(X)))  # Fit the Bayesian Stochastic Volatility EKF-HMC model
        h = np.mean(samples, axis=0)  # Calculate mean of samples
        sigma_weights = np.sqrt(np.exp(h[0] + h[1]))  # Define sigma weights
        return self.svm.predict(X, sample_weight=sigma_weights)  # Predict using SVR model with sigma weights

# Instantiate and fit the hybrid model
hybrid_model = HybridModel()
hybrid_model.fit(X, y, log_returns)

# Predictions
predictions = hybrid_model.predict(X)
print("Predictions:", predictions)


[*********************100%%**********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


X shape: (244, 1), y shape: (244,)
Initial X: [[-0.01403967]
 [-0.0039223 ]
 [-0.00097102]
 [ 0.02289494]
 [-0.02000714]]
Initial y: [0.01862592 0.01864767 0.01809953 0.01864491 0.01853943]
Samples summary:
[[0.  0.9 0.1]
 [0.  0.9 0.1]
 [0.  0.9 0.1]
 ...
 [0.  0.9 0.1]
 [0.  0.9 0.1]
 [0.  0.9 0.1]]


IndexError: tuple index out of range